# Hypothesis

H0: A CNN model trained for skin cancer detection of moles looks at other areas of the image rather than the mole itself.

H1: A CNN model trained for skin cancer detection of moles looks at the mole itself rather than other areas of the image.

In [6]:
#!pip install opencv-python

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
import kagglehub
from tqdm import tqdm

# Data setup
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
df = pd.read_csv(os.path.join(path, "HAM10000_metadata.csv"))
# Concept: mole (melanoma, label==1); benign lesions, label==0
df["label"] = df["dx"].apply(lambda x: 1 if x == "mel" else 0)
df["image_path"] = df["image_id"].apply(
    lambda x: os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg")
    if os.path.exists(os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg"))
    else os.path.join(path, "HAM10000_images_part_2", f"{x}.jpg")
)
train_df = df.iloc[:5000]
test_df = df.iloc[5000:]
train_df, val_df = train_df.iloc[:4000], train_df.iloc[4000:]

# Create a cache directory for transformed images
cache_dir = "processed_ham10000"
os.makedirs(cache_dir, exist_ok=True)

# Dataset with caching and progress indication for pre-caching
class HAM10000Dataset(Dataset):
    def __init__(self, df, transform=None, cache=True, cache_dir=cache_dir, precache=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.cache = cache
        self.cache_dir = cache_dir

        if self.cache and precache:
            print("Pre-caching images...")
            for i in tqdm(range(len(self.df)), desc="Caching images"):
                _ = self.__getitem__(i)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image_id = row["image_id"]
        label = torch.tensor(row["label"], dtype=torch.long)
        cache_path = os.path.join(self.cache_dir, f"{image_id}.pt")
        if self.cache and os.path.exists(cache_path):
            image = torch.load(cache_path, map_location="cpu", weights_only=False)
        else:
            img_path = row["image_path"]
            image = read_image(img_path).float() / 255.0
            if self.transform:
                image = self.transform(image)
            if self.cache:
                torch.save(image, cache_path)
        return image, label

# Define transform and create datasets with pre-caching enabled
transform = transforms.Compose([transforms.Resize((224, 224))])
train_dataset = HAM10000Dataset(train_df, transform, precache=True)
val_dataset   = HAM10000Dataset(val_df, transform, precache=True)
test_dataset  = HAM10000Dataset(test_df, transform, precache=True)

# Create DataLoaders with progress wrapped in training loops
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pretrained ResNet-50 and freeze all layers except layer4
base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
for name, param in base_model.named_parameters():
    param.requires_grad = ("layer4" in name)

# Custom model: ResNet-50 base with GAP and a two-layer feedforward network
class CustomResNet(nn.Module):
    def __init__(self, base_model):
        super(CustomResNet, self).__init__()
        self.base = base_model
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(base_model.fc.in_features, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 1)
    def forward(self, x):
        x = self.base.conv1(x)
        x = self.base.bn1(x)
        x = self.base.relu(x)
        x = self.base.maxpool(x)
        x = self.base.layer1(x)
        x = self.base.layer2(x)
        x = self.base.layer3(x)
        x = self.base.layer4(x)
        x = self.base.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomResNet(base_model).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, count = 0, 0, 0
    for imgs, labels in tqdm(loader, desc="Evaluating", leave=False):
        imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        total_loss += loss.item() * imgs.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).long()
        correct += (preds == labels.long()).sum().item()
        count += imgs.size(0)
    return total_loss / count, correct / count

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} Training", leave=False):
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * imgs.size(0)
        train_loss = epoch_loss / len(train_loader.dataset)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}: Train {train_loss:.4f} | Val {val_loss:.4f} | Val Acc {val_acc:.4f}")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss {test_loss:.4f} | Test Acc {test_acc:.4f}")

# Activation extractor from "base.layer4"
class ActivationExtractor(nn.Module):
    def __init__(self, model, layer_name):
        super().__init__()
        self.model = model
        self.layer_name = layer_name
        self.activations = []
    def hook_fn(self, module, input, output):
        self.activations.append(output.detach())
    def get_activations(self, x):
        self.activations = []
        handle = dict([*self.model.named_modules()])[self.layer_name].register_forward_hook(self.hook_fn)
        _ = self.model(x)
        handle.remove()
        return self.activations[0]

extractor = ActivationExtractor(model, "base.layer4")

def extract_acts(dataset, indices):
    acts = []
    for idx in indices:
        img, _ = dataset[idx]
        img = img.unsqueeze(0).to(device)
        act = extractor.get_activations(img)
        acts.append(act.view(act.size(1), -1).mean(dim=1).cpu().numpy())
    return np.array(acts)

# TCAV: Compare concept (mole, label==1) vs. random (benign, label==0)
N = 10
concept_pool = train_df[train_df["label"] == 1].index.tolist()
random_pool = train_df[train_df["label"] == 0].index.tolist()
concept_sample_size = min(30, len(concept_pool))
fixed_concept_indices = random.sample(concept_pool, concept_sample_size)
concept_acts = extract_acts(train_dataset, fixed_concept_indices)

tcav_scores = []
for i in range(N):
    rand_indices = random.sample(random_pool, concept_sample_size)
    random_acts = extract_acts(train_dataset, rand_indices)
    X = np.vstack([concept_acts, random_acts])
    y = [1] * len(concept_acts) + [0] * len(random_acts)
    clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
    clf.fit(X, y)
    scores = clf.decision_function(concept_acts)
    tcav_scores.append(np.mean(scores > 0))

plt.figure(figsize=(8, 5))
plt.boxplot(tcav_scores)
plt.title("TCAV Scores (Mole vs. Benign)")
plt.ylabel("TCAV Score")
plt.xticks([1], ["TCAV"])
plt.show()

# Visualize sample concept images (moles)
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for ax, idx in zip(axs, fixed_concept_indices[:5]):
    img, _ = train_dataset[idx]
    ax.imshow(img.permute(1, 2, 0).cpu().numpy())
    ax.axis("off")
plt.suptitle("Sample Mole Images (Concept)")
plt.show()

# Feature map visualization from "base.layer4" for a sample image
def visualize_activation(image, layer="base.layer4", num_maps=5):
    model.eval()
    activations = []
    def hook_fn(module, input, output):
        activations.append(output.detach().cpu())
    handle = dict([*model.named_modules()])[layer].register_forward_hook(hook_fn)
    _ = model(image.unsqueeze(0).to(device))
    handle.remove()
    act = activations[0].squeeze(0)
    fig, axs = plt.subplots(1, num_maps, figsize=(15, 3))
    for i in range(num_maps):
        axs[i].imshow(act[i].numpy(), cmap="viridis")
        axs[i].axis("off")
    plt.suptitle("Feature Maps from Base.Layer4")
    plt.show()

sample_img, _ = train_dataset[0]
visualize_activation(sample_img, "base.layer4", num_maps=5)

# GradCAM visualization using "base.layer4"
def grad_cam(model, image, target_layer="base.layer4", target_class=1):
    model.eval()
    activations, gradients = {}, {}
    def forward_hook(module, input, output):
        activations["value"] = output
    def backward_hook(module, grad_in, grad_out):
        gradients["value"] = grad_out[0]
    target_module = dict([*model.named_modules()])[target_layer]
    h1 = target_module.register_forward_hook(forward_hook)
    h2 = target_module.register_backward_hook(backward_hook)
    image = image.unsqueeze(0).to(device)
    output = model(image)
    score = output if target_class == 1 else -output
    model.zero_grad()
    score.backward()
    act = activations["value"].detach()
    grad = gradients["value"].detach()
    weights = grad.mean(dim=(2, 3), keepdim=True)
    cam = torch.relu((weights * act).sum(dim=1, keepdim=True))
    cam = torch.nn.functional.interpolate(cam, size=(224, 224), mode="bilinear", align_corners=False)
    cam = cam.squeeze().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    h1.remove(); h2.remove()
    return cam

heatmap = grad_cam(model, sample_img, "base.layer4", target_class=1)
img_np = sample_img.permute(1, 2, 0).cpu().numpy()
plt.imshow(img_np)
plt.imshow(heatmap, cmap="jet", alpha=0.5)
plt.title("GradCAM - Mole (Concept)")
plt.axis("off")
plt.show()

# Integration for Assignment:
#
# H0: The pretrained ResNet-50 model (with only its last bottleneck block and a new head unfrozen)
#     does not significantly utilize the "mole" concept in its predictions.
# H1: The model's predictions are significantly influenced by the "mole" concept.
#
# Approach:
# 1. Use a frozen ResNet-50 base (except layer4) and add a two-layer feedforward network.
# 2. Compute TCAV scores by contrasting activations from the concept set (mole images)
#    with those from a random set (benign lesions).
# 3. Visualize the distribution of TCAV scores, sample concept images, internal feature maps,
#    and GradCAM overlays to provide both quantitative and qualitative insights.
#
# The dataset class caches transformed images in 'processed_ham10000' and shows progress
# during pre-caching and data loading via tqdm.

Pre-caching images...


Caching images: 100%|█████████████████████| 4000/4000 [00:03<00:00, 1332.08it/s]


Pre-caching images...


Caching images: 100%|█████████████████████| 1000/1000 [00:00<00:00, 1310.40it/s]


Pre-caching images...


Caching images: 100%|██████████████████████| 5015/5015 [00:39<00:00, 127.05it/s]
                                                                                

Epoch 1: Train 0.3668 | Val 0.0141 | Val Acc 0.9990


Epoch 2 Training:   2%|▍                        | 2/125 [00:10<10:53,  5.32s/it]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
import kagglehub
import random

# Data setup
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
df = pd.read_csv(os.path.join(path, "HAM10000_metadata.csv"))
# Concept: mole (melanoma, label==1); benign lesions, label==0
df["label"] = df["dx"].apply(lambda x: 1 if x == "mel" else 0)
df["image_path"] = df["image_id"].apply(
    lambda x: os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg")
    if os.path.exists(os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg"))
    else os.path.join(path, "HAM10000_images_part_2", f"{x}.jpg")
)
train_df = df.iloc[:5000]
test_df = df.iloc[5000:]
train_df, val_df = train_df.iloc[:4000], train_df.iloc[4000:]

# Create a cache directory for transformed images
cache_dir = "processed_ham10000"
os.makedirs(cache_dir, exist_ok=True)

# Dataset with caching to speed up future loads
class HAM10000Dataset(Dataset):
    def __init__(self, df, transform=None, cache=True, cache_dir=cache_dir):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.cache = cache
        self.cache_dir = cache_dir
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image_id = row["image_id"]
        label = torch.tensor(row["label"], dtype=torch.long)
        cache_path = os.path.join(self.cache_dir, f"{image_id}.pt")
        if self.cache and os.path.exists(cache_path):
            image = torch.load(cache_path, map_location="cpu", weights_only=False)  # weights_only=False is default but explicit
        else:
            img_path = row["image_path"]
            image = read_image(img_path).float() / 255.0
            if self.transform:
                image = self.transform(image)
            if self.cache:
                torch.save(image, cache_path)
        return image, label

transform = transforms.Compose([transforms.Resize((224, 224))])
train_dataset = HAM10000Dataset(train_df, transform)
val_dataset = HAM10000Dataset(val_df, transform)
test_dataset = HAM10000Dataset(test_df, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pretrained ResNet-50 and freeze all layers except the last bottleneck block (layer4)
base_model = models.resnet50(weights=True)
for name, param in base_model.named_parameters():
    if "layer4" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Custom model: ResNet-50 base with GAP and a two-layer feedforward network
class CustomResNet(nn.Module):
    def __init__(self, base_model):
        super(CustomResNet, self).__init__()
        self.base = base_model  # pretrained ResNet-50 backbone
        # New head: uses the features from ResNet's GAP
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(base_model.fc.in_features, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 1)
    def forward(self, x):
        x = self.base.conv1(x)
        x = self.base.bn1(x)
        x = self.base.relu(x)
        x = self.base.maxpool(x)
        x = self.base.layer1(x)
        x = self.base.layer2(x)
        x = self.base.layer3(x)
        x = self.base.layer4(x)
        x = self.base.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomResNet(base_model).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, count = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            count += imgs.size(0)
    return total_loss / count, correct / count

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * imgs.size(0)
        train_loss = epoch_loss / len(train_loader.dataset)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}: Train {train_loss:.4f} | Val {val_loss:.4f} | Val Acc {val_acc:.4f}")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss {test_loss:.4f} | Test Acc {test_acc:.4f}")

# Activation extractor from the target layer ("base.layer4")
class ActivationExtractor(nn.Module):
    def __init__(self, model, layer_name):
        super().__init__()
        self.model = model
        self.layer_name = layer_name
        self.activations = []
    def hook_fn(self, module, input, output):
        self.activations.append(output.detach())
    def get_activations(self, x):
        self.activations = []
        handle = dict([*self.model.named_modules()])[self.layer_name].register_forward_hook(self.hook_fn)
        _ = self.model(x)
        handle.remove()
        return self.activations[0]

extractor = ActivationExtractor(model, "base.layer4")

# Extract flattened activations (averaging spatial dimensions)
def extract_acts(dataset, indices):
    acts = []
    for idx in indices:
        img, _ = dataset[idx]
        img = img.unsqueeze(0).to(device)
        act = extractor.get_activations(img)  # shape: [1, C, H, W]
        acts.append(act.view(act.size(1), -1).mean(dim=1).cpu().numpy())
    return np.array(acts)

# TCAV: Compare concept (mole, label==1) vs. random (benign, label==0)
N = 10  # number of random trials
concept_pool = train_df[train_df["label"] == 1].index.tolist()
random_pool = train_df[train_df["label"] == 0].index.tolist()
concept_sample_size = min(30, len(concept_pool))
fixed_concept_indices = random.sample(concept_pool, concept_sample_size)
concept_acts = extract_acts(train_dataset, fixed_concept_indices)

tcav_scores = []
for i in range(N):
    rand_indices = random.sample(random_pool, concept_sample_size)
    random_acts = extract_acts(train_dataset, rand_indices)
    X = np.vstack([concept_acts, random_acts])
    y = np.array([1] * len(concept_acts) + [0] * len(random_acts))
    clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
    clf.fit(X, y)
    scores = clf.decision_function(concept_acts)
    tcav_score = np.mean(scores > 0)
    tcav_scores.append(tcav_score)

plt.figure(figsize=(8, 5))
plt.boxplot(tcav_scores)
plt.title("TCAV Scores (Mole vs. Benign)")
plt.ylabel("TCAV Score")
plt.xticks([1], ["TCAV"])
plt.show()

# Visualize sample concept images (moles)
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
sample_concept_indices = fixed_concept_indices[:5]
for ax, idx in zip(axs, sample_concept_indices):
    img, _ = train_dataset[idx]
    ax.imshow(img.permute(1, 2, 0).cpu().numpy())
    ax.axis("off")
plt.suptitle("Sample Mole Images (Concept)")
plt.show()

# Feature map visualization from "base.layer4" for a sample image
def visualize_activation(image, layer="base.layer4", num_maps=5):
    model.eval()
    activations = []
    def hook_fn(module, input, output):
        activations.append(output.detach().cpu())
    handle = dict([*model.named_modules()])[layer].register_forward_hook(hook_fn)
    _ = model(image.unsqueeze(0).to(device))
    handle.remove()
    act = activations[0].squeeze(0)  # shape: [C, H, W]
    fig, axs = plt.subplots(1, num_maps, figsize=(15, 3))
    for i in range(num_maps):
        axs[i].imshow(act[i].numpy(), cmap="viridis")
        axs[i].axis("off")
    plt.suptitle("Feature Maps from Base.Layer4")
    plt.show()

sample_img, _ = train_dataset[0]
visualize_activation(sample_img, "base.layer4", num_maps=5)

# GradCAM visualization using "base.layer4"
def grad_cam(model, image, target_layer="base.layer4", target_class=1):
    model.eval()
    activations, gradients = {}, {}
    def forward_hook(module, input, output):
        activations["value"] = output
    def backward_hook(module, grad_in, grad_out):
        gradients["value"] = grad_out[0]
    target_module = dict([*model.named_modules()])[target_layer]
    h1 = target_module.register_forward_hook(forward_hook)
    h2 = target_module.register_backward_hook(backward_hook)
    image = image.unsqueeze(0).to(device)
    output = model(image)
    score = output if target_class == 1 else -output
    model.zero_grad()
    score.backward()
    act = activations["value"].detach()
    grad = gradients["value"].detach()
    weights = grad.mean(dim=(2, 3), keepdim=True)
    cam = torch.relu((weights * act).sum(dim=1, keepdim=True))
    cam = torch.nn.functional.interpolate(cam, size=(224, 224), mode="bilinear", align_corners=False)
    cam = cam.squeeze().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    h1.remove(); h2.remove()
    return cam

heatmap = grad_cam(model, sample_img, "base.layer4", target_class=1)
img_np = sample_img.permute(1, 2, 0).cpu().numpy()
plt.imshow(img_np)
plt.imshow(heatmap, cmap="jet", alpha=0.5)
plt.title("GradCAM - Mole (Concept)")
plt.axis("off")
plt.show()

# Integration for Assignment:
#
# H0: The pretrained ResNet-50 model (with only its last bottleneck block and a new head unfrozen)
#     does not significantly utilize the "mole" concept in its predictions.
# H1: The model's predictions are significantly influenced by the "mole" concept.
#
# Approach:
# 1. Use a frozen ResNet-50 base (except layer4) and add a two-layer feedforward network.
# 2. Compute TCAV scores by contrasting activations from the concept set (mole images)
#    with those from a random set (benign lesions).
# 3. Visualize the distribution of TCAV scores, sample concept images, internal feature maps,
#    and GradCAM overlays to provide both quantitative and qualitative insights.
#
# The dataset class caches transformed images in 'processed_ham10000' for faster loading on future runs.

/Users/ryan/Projects/XAI/Xenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.stats import permutation_test
from sklearn.linear_model import SGDClassifier
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import kagglehub

# Data
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
print("Dataset path:", path)
df = pd.read_csv(os.path.join(path, "HAM10000_metadata.csv"))
df["label"] = df["dx"].apply(lambda x: 1 if x == "mel" else 0)
df["image_path"] = df["image_id"].apply(
    lambda x: os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg")
    if os.path.exists(os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg"))
    else os.path.join(path, "HAM10000_images_part_2", f"{x}.jpg")
)
train_df = df.iloc[:5000]
test_df = df.iloc[5000:]
train_df, val_df = train_df.iloc[:4000], train_df.iloc[4000:]

class HAM10000Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image_path"]
        image = read_image(img_path).float() / 255.0
        label = torch.tensor(self.df.iloc[idx]["label"], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([transforms.Resize((224, 224))])
train_dataset = HAM10000Dataset(train_df, transform)
val_dataset = HAM10000Dataset(val_df, transform)
test_dataset = HAM10000Dataset(test_df, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# CNN with BN and dropout
class SkinCancerCNN(nn.Module):
    def __init__(self):
        super(SkinCancerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(64 * 28 * 28, 1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SkinCancerCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, count = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            count += imgs.size(0)
    return total_loss / count, correct / count

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * imgs.size(0)
        train_loss = epoch_loss / len(train_loader.dataset)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f} | Val Acc {val_acc:.4f}")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss {test_loss:.4f} | Test Acc {test_acc:.4f}")

# TCAV
class ActivationExtractor(nn.Module):
    def __init__(self, model, layer):
        super().__init__()
        self.model = model
        self.layer = layer
        self.activations = []
    def hook_fn(self, module, input, output):
        self.activations.append(output.detach())
    def get_activations(self, x):
        self.activations = []
        handle = getattr(self.model, self.layer).register_forward_hook(self.hook_fn)
        _ = self.model(x)
        handle.remove()
        return self.activations[0].cpu().numpy()

extractor = ActivationExtractor(model, 'conv3')
concept_acts, non_concept_acts = [], []
for imgs, labels in train_loader:
    imgs = imgs.to(device)
    acts = extractor.get_activations(imgs)
    labels_np = labels.cpu().numpy()
    for i in range(len(labels_np)):
        if labels_np[i] == 1:
            concept_acts.append(acts[i])
        else:
            non_concept_acts.append(acts[i])
concept_acts = np.array(concept_acts).reshape(len(concept_acts), -1)
non_concept_acts = np.array(non_concept_acts).reshape(len(non_concept_acts), -1)
cav_model = SGDClassifier()
cav_labels = np.array([1] * len(concept_acts) + [0] * len(non_concept_acts))
cav_data = np.vstack((concept_acts, non_concept_acts))
cav_model.fit(cav_data, cav_labels)
tcav_scores = cav_model.decision_function(concept_acts)
tcav_score = np.mean(tcav_scores > 0)
print(f"TCAV Score: {tcav_score:.4f}")
def diff_mean(a, b):
    return np.mean(a) - np.mean(b)
perm_result = permutation_test(
    (concept_acts.flatten(), non_concept_acts.flatten()),
    statistic=diff_mean,
    vectorized=False,
    n_resamples=1000,
    alternative="greater",
)
print(f"Permutation Test p-value: {perm_result.pvalue:.4f}")

# Visualize sample concept images
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
concept_indices = train_df[train_df["label"] == 1].index[:5]
for ax, idx in zip(axs, concept_indices):
    img, _ = train_dataset[idx]
    ax.imshow(img.permute(1, 2, 0).numpy())
    ax.axis("off")
plt.suptitle("Concept Images")
plt.show()

# Feature map visualization
def visualize_activation(model, image, layer="conv3"):
    model.eval()
    activations = []
    def hook_fn(module, input, output):
        activations.append(output.detach().cpu())
    handle = getattr(model, layer).register_forward_hook(hook_fn)
    _ = model(image.unsqueeze(0).to(device))
    handle.remove()
    act = activations[0].squeeze(0)
    num_maps = act.shape[0]
    fig, axs = plt.subplots(1, min(num_maps, 5), figsize=(15, 3))
    for i in range(min(num_maps, 5)):
        axs[i].imshow(act[i].numpy(), cmap="viridis")
        axs[i].axis("off")
    plt.suptitle("Feature Maps")
    plt.show()

sample_img, _ = train_dataset[0]
visualize_activation(model, sample_img, "conv3")

# GradCAM
def grad_cam(model, image, target_layer="conv3", target_class=1):
    model.eval()
    activations, gradients = {}, {}
    def forward_hook(module, input, output):
        activations["value"] = output
    def backward_hook(module, grad_in, grad_out):
        gradients["value"] = grad_out[0]
    target_module = getattr(model, target_layer)
    h1 = target_module.register_forward_hook(forward_hook)
    h2 = target_module.register_backward_hook(backward_hook)
    image = image.unsqueeze(0).to(device)
    output = model(image)
    score = output if target_class == 1 else -output
    model.zero_grad()
    score.backward()
    act = activations["value"].detach()
    grad = gradients["value"].detach()
    weights = grad.mean(dim=(2, 3), keepdim=True)
    cam = torch.relu((weights * act).sum(dim=1, keepdim=True))
    cam = torch.nn.functional.interpolate(cam, size=(224, 224), mode="bilinear", align_corners=False)
    cam = cam.squeeze().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    h1.remove(); h2.remove()
    return cam

heatmap = grad_cam(model, sample_img, "conv3", target_class=1)
img_np = sample_img.permute(1, 2, 0).numpy()
plt.imshow(img_np)
plt.imshow(heatmap, cmap="jet", alpha=0.5)
plt.title("GradCAM")
plt.axis("off")
plt.show()

Dataset path: /Users/ryan/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2
Epoch 1: Train Loss 1.0060 | Val Loss 1.0641 | Val Acc 0.5820
Epoch 2: Train Loss 0.5738 | Val Loss 0.0325 | Val Acc 0.9900
Epoch 3: Train Loss 0.5380 | Val Loss 0.0289 | Val Acc 0.9890
Epoch 4: Train Loss 0.4610 | Val Loss 0.1205 | Val Acc 0.9490
Epoch 5: Train Loss 0.4126 | Val Loss 0.0033 | Val Acc 0.9990
Test Loss 0.6973 | Test Acc 0.7093
TCAV Score: 0.9188


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import os
from scipy.stats import permutation_test
from sklearn.linear_model import SGDClassifier
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import kagglehub

# Data setup
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
print("Dataset path:", path)
df = pd.read_csv(os.path.join(path, "HAM10000_metadata.csv"))
df["label"] = df["dx"].apply(lambda x: 1 if x == "mel" else 0)
df["image_path"] = df["image_id"].apply(lambda x: os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg")
    if os.path.exists(os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg"))
    else os.path.join(path, "HAM10000_images_part_2", f"{x}.jpg"))
train_df = df.iloc[:5000]
test_df = df.iloc[5000:]
train_df, val_df = train_df.iloc[:4000], train_df.iloc[4000:]

# Dataset
class HAM10000Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image_path"]
        image = read_image(img_path).float() / 255.0
        label = torch.tensor(self.df.iloc[idx]["label"], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([transforms.Resize((224, 224))])
train_dataset = HAM10000Dataset(train_df, transform)
val_dataset = HAM10000Dataset(val_df, transform)
test_dataset = HAM10000Dataset(test_df, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model
class SkinCancerCNN(nn.Module):
    def __init__(self):
        super(SkinCancerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 28 * 28, 1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SkinCancerCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Evaluation
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, correct, count = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * imgs.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            count += imgs.size(0)
    return total_loss / count, correct / count

# Training
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)
        train_loss = running_loss / len(train_loader.dataset)
        val_loss, val_acc = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}: Train {train_loss:.4f}, Val {val_loss:.4f}, Val Acc {val_acc:.4f}")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test {test_loss:.4f}, Test Acc {test_acc:.4f}")

# TCAV
class ActivationExtractor(nn.Module):
    def __init__(self, model, layer):
        super().__init__()
        self.model = model
        self.layer = layer
        self.activations = []
    def forward_hook(self, module, input, output):
        self.activations.append(output.detach())
    def get_activations(self, x):
        self.activations = []
        handle = getattr(self.model, self.layer).register_forward_hook(self.forward_hook)
        _ = self.model(x)
        handle.remove()
        return self.activations[0].cpu().numpy()

extractor = ActivationExtractor(model, 'conv3')
concept_acts, non_concept_acts = [], []
for imgs, labels in train_loader:
    imgs = imgs.to(device)
    labels_np = labels.cpu().numpy()
    acts = extractor.get_activations(imgs)
    concept_acts.extend(acts[labels_np == 1])
    non_concept_acts.extend(acts[labels_np == 0])
concept_acts = np.array(concept_acts).reshape(len(concept_acts), -1)
non_concept_acts = np.array(non_concept_acts).reshape(len(non_concept_acts), -1)

cav_model = SGDClassifier()
cav_labels = np.array([1] * len(concept_acts) + [0] * len(non_concept_acts))
cav_data = np.vstack((concept_acts, non_concept_acts))
cav_model.fit(cav_data, cav_labels)
tcav_scores = cav_model.decision_function(concept_acts)
tcav_score = np.mean(tcav_scores > 0)
print(f"TCAV Score: {tcav_score}")

def permutation_test_func(a, b):
    return np.mean(a) - np.mean(b)
perm_result = permutation_test((concept_acts.flatten(), non_concept_acts.flatten()),
                               statistic=permutation_test_func, alternative='greater')
print(f"Permutation Test p-value: {perm_result.pvalue}")

Dataset path: /Users/ryan/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2
Epoch 1: Train 0.5294, Val 0.3571, Val Acc 0.8990
Epoch 2: Train 0.4276, Val 0.2209, Val Acc 0.9350
Epoch 3: Train 0.4117, Val 0.0379, Val Acc 0.9970
Epoch 4: Train 0.3815, Val 0.2909, Val Acc 0.8920
Epoch 5: Train 0.3703, Val 0.1911, Val Acc 0.9410
Test 0.6835, Test Acc 0.6931
TCAV Score: 0.6393146979260595


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from scipy.stats import permutation_test
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image

# Install dependencies as needed:
# pip install kagglehub
import kagglehub

# Download latest version of HAM10000 dataset
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

# Load HAM10000 metadata
df = pd.read_csv(os.path.join(path, "HAM10000_metadata.csv"))

# Map 'mel' as cancerous (1), others as non-cancerous (0)
df["label"] = df["dx"].apply(lambda x: 1 if x == "mel" else 0)

# Assign correct image paths while keeping original folder names
df["image_path"] = df["image_id"].apply(lambda x: os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg")
    if os.path.exists(os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg"))
    else os.path.join(path, "HAM10000_images_part_2", f"{x}.jpg"))

# Split into training and testing
train_df = df.iloc[:5000]
test_df = df.iloc[5000:]

# Image Dataset class
class HAM10000Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image_path"]
        image = read_image(img_path).float() / 255.0
        label = torch.tensor(self.df.iloc[idx]["label"], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
])

train_dataset = HAM10000Dataset(train_df, transform=transform)
test_dataset = HAM10000Dataset(test_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Simple CNN Model
class SkinCancerCNN(nn.Module):
    def __init__(self):
        super(SkinCancerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(64 * 28 * 28, 1)  # Binary classification
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return torch.sigmoid(x)

# Initialize model, loss function, optimizer
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SkinCancerCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_model(model, train_loader, criterion, optimizer)

# TCAV: Extract activations
class ActivationExtractor(nn.Module):
    def __init__(self, model, layer):
        super().__init__()
        self.model = model
        self.layer = layer
        self.activations = []

    def forward_hook(self, module, input, output):
        self.activations.append(output.detach())

    def get_activations(self, x):
        self.activations = []
        handle = getattr(self.model, self.layer).register_forward_hook(self.forward_hook)
        _ = self.model(x)
        handle.remove()
        return self.activations[0].cpu().numpy()

extractor = ActivationExtractor(model, 'conv3')

# Get concept (mole) and non-concept (background) activations
concept_activations, non_concept_activations = [], []

for images, labels in dataloader:
    images = images.to(device)
    activations = extractor.get_activations(images)
    concept_activations.extend(activations[labels == 1])  # Assume 1 is 'mole'
    non_concept_activations.extend(activations[labels == 0])  # Assume 0 is 'background'

concept_activations = np.array(concept_activations).reshape(len(concept_activations), -1)
non_concept_activations = np.array(non_concept_activations).reshape(len(non_concept_activations), -1)

# Train Concept Activation Vectors (CAVs)
cav_model = SGDClassifier()
cav_labels = np.array([1] * len(concept_activations) + [0] * len(non_concept_activations))
cav_data = np.vstack((concept_activations, non_concept_activations))
cav_model.fit(cav_data, cav_labels)

# Compute TCAV score
tcav_scores = cav_model.decision_function(concept_activations)
tcav_score = np.mean(tcav_scores > 0)

print(f"TCAV Score (higher = more reliance on concept): {tcav_score}")

# Statistical Testing (Permutation Test)
def permutation_test_func(a, b):
    return np.mean(a) - np.mean(b)

perm_result = permutation_test((concept_activations.flatten(), non_concept_activations.flatten()),
                               statistic=permutation_test_func, alternative='greater')

print(f"Permutation Test p-value: {perm_result.pvalue}")


Path to dataset files: /Users/ryan/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2
Epoch 1/5, Loss: 0.6931471824645996
Epoch 2/5, Loss: 0.6931471824645996
Epoch 3/5, Loss: 0.6931471824645996
Epoch 4/5, Loss: 0.6931471824645996
Epoch 5/5, Loss: 0.6931471824645996


NameError: name 'dataloader' is not defined

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from scipy.stats import permutation_test
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image

# Install dependencies as needed:
# pip install kagglehub
import kagglehub

# Download latest version of HAM10000 dataset
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

# Load HAM10000 metadata
df = pd.read_csv(os.path.join(path, "HAM10000_metadata.csv"))

# Map 'mel' as cancerous (1), others as non-cancerous (0)
df["label"] = df["dx"].apply(lambda x: 1 if x == "mel" else 0)

# Assign correct image paths while keeping original folder names
df["image_path"] = df["image_id"].apply(lambda x: os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg")
    if os.path.exists(os.path.join(path, "HAM10000_images_part_1", f"{x}.jpg"))
    else os.path.join(path, "HAM10000_images_part_2", f"{x}.jpg"))

# Split into training and testing
train_df = df.iloc[:5000]
test_df = df.iloc[5000:]

# Image Dataset class
class HAM10000Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]["image_path"]
        image = read_image(img_path).float() / 255.0
        label = torch.tensor(self.df.iloc[idx]["label"], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
])

train_dataset = HAM10000Dataset(train_df, transform=transform)
test_dataset = HAM10000Dataset(test_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Simple CNN Model
class SkinCancerCNN(nn.Module):
    def __init__(self):
        super(SkinCancerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(64 * 28 * 28, 2)  # Binary classification
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Initialize model, loss function, optimizer
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = SkinCancerCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_model(model, train_loader, criterion, optimizer)

# TCAV and Statistical Testing omitted for brevity, apply same logic with new dataset


100%|██████████████████████████████████████| 5.20G/5.20G [02:15<00:00, 41.1MB/s]

Extracting files...


Path to dataset files: /Users/ryan/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2
Epoch 1/5, Loss: 0.1766643077135086
Epoch 2/5, Loss: 0.6814433336257935
Epoch 3/5, Loss: 0.3563919961452484
Epoch 4/5, Loss: 0.9468994140625
Epoch 5/5, Loss: 0.36127737164497375


In [10]:
# TCAV: Extract activations
class ActivationExtractor(nn.Module):
    def __init__(self, model, layer):
        super().__init__()
        self.model = model
        self.layer = layer
        self.activations = []

    def forward_hook(self, module, input, output):
        self.activations.append(output.detach())

    def get_activations(self, x):
        self.activations = []
        handle = getattr(self.model, self.layer).register_forward_hook(self.forward_hook)
        _ = self.model(x)
        handle.remove()
        return self.activations[0].cpu().numpy()

extractor = ActivationExtractor(model, 'conv3')

# Get concept (mole) and non-concept (background) activations
concept_activations, non_concept_activations = [], []

for images, labels in dataloader:
    images = images.to(device)
    activations = extractor.get_activations(images)
    concept_activations.extend(activations[labels == 1])  # Assume 1 is 'mole'
    non_concept_activations.extend(activations[labels == 0])  # Assume 0 is 'background'

concept_activations = np.array(concept_activations).reshape(len(concept_activations), -1)
non_concept_activations = np.array(non_concept_activations).reshape(len(non_concept_activations), -1)

# Train Concept Activation Vectors (CAVs)
cav_model = SGDClassifier()
cav_labels = np.array([1] * len(concept_activations) + [0] * len(non_concept_activations))
cav_data = np.vstack((concept_activations, non_concept_activations))
cav_model.fit(cav_data, cav_labels)

# Compute TCAV score
tcav_scores = cav_model.decision_function(concept_activations)
tcav_score = np.mean(tcav_scores > 0)

print(f"TCAV Score (higher = more reliance on concept): {tcav_score}")

# Statistical Testing (Permutation Test)
def permutation_test_func(a, b):
    return np.mean(a) - np.mean(b)

perm_result = permutation_test((concept_activations.flatten(), non_concept_activations.flatten()),
                               statistic=permutation_test_func, alternative='greater')

print(f"Permutation Test p-value: {perm_result.pvalue}")


NameError: name 'dataloader' is not defined